In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用云剖析器(Profile model training performance using Cloud Profiler)

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom_training_tensorboard_profiler.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fcustom%2Fcustom_training_tensorboard_profiler.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/custom/custom_training_tensorboard_profiler.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom_training_tensorboard_profiler.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

Cloud Profiler 允许您监控和优化模型训练性能，帮助您了解训练操作的资源消耗。本教程演示了如何启用 Cloud Profiler，以便您可以调试自定义训练作业的模型训练性能。

了解更多关于[Cloud Profiler](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-profiler)。

### 目标

在本教程中，您将学习如何为自定义训练作业启用云分析器。

本教程使用以下 Google Cloud AI 服务：

- `Vertex AI Training`
- `Vertex AI TensorBoard`

执行的步骤包括：

- 设置服务帐号和云存储桶
- 创建一个 TensorBoard 实例
- 创建并运行一个自定义训练作业
- 查看云分析器仪表板

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[mnist数据集](https://www.tensorflow.org/datasets/catalog/mnist)。

成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
定价](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预计使用情况生成成本估算。

开始吧

### 为 Python 安装 Vertex AI SDK 和其他必需的软件包

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform --quiet

### 重新启动运行时（仅适用于Colab）

要使用新安装的包，必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核即将重新启动。请等待重新启动完成后再继续下一步。 ⚠️</b>
</div>

### 在谷歌 Colab 上验证您的笔记本环境

在谷歌 Colab 上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置谷歌云项目信息并初始化用于 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须拥有现有的谷歌云项目，并 [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于 [设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment) 的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶 

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化用于 Python 的 Vertex AI SDK。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### 设置服务账号和权限

将使用服务账号来创建自定义训练作业。如果您不想使用项目的 Compute Engine 服务账号，请将SERVICE_ACCOUNT设置为另一个服务账号ID。您可以按照[说明](https://cloud.google.com/iam/docs/creating-managing-service-accounts#creating)来创建服务账号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

为Vertex AI Pipelines设置服务帐户访问权限

运行以下命令，将您的服务帐户访问权限授予到在上一步中创建的存储桶中读取和写入管道工件。您只需要每个服务帐户运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 启用制品注册表 API

首先，您必须为您的项目启用制品注册表 API 服务。

了解更多关于[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

在协作环境中设置项目（仅限Colab）

In [ ]:
if IS_COLAB:
    ! gcloud config set project $PROJECT_ID

In [ ]:
# Enable Artifact Registry API
! gcloud services enable artifactregistry.googleapis.com --quiet

### 创建一个 TensorBoard 实例

在展示 Vertex AI TensorBoard 实验之前，必须创建一个 Vertex AI TensorBoard 实例，这是一个将您的 Vertex AI TensorBoard 实验存储在某个区域的资源。您可以在一个项目中创建多个实例。您可以使用命令 `gcloud ai tensorboards list` 来获取您现有的 TensorBoard 实例列表。

设置您的TensorBoard实例显示名称

In [ ]:
TENSORBOARD_NAME = "your-tensorboard-unique"  # @param {type:"string"}

#### 创建一个TensorBoard实例

如果您还没有TensorBoard实例，请通过运行以下单元格来创建一个：

In [ ]:
tensorboard = aiplatform.Tensorboard.create(
    display_name=TENSORBOARD_NAME, project=PROJECT_ID, location=LOCATION
)

TENSORBOARD_INSTANCE_NAME = tensorboard.resource_name

print("TensorBoard instance name:", TENSORBOARD_INSTANCE_NAME)

训练模型

使用您自定义的训练代码训练模型时，选择以下其中一种选项：

- **预构建容器**：将您的自定义训练代码作为 Python 包加载到 Google Cloud 的预构建容器镜像中。

- **自定义容器**：创建包含您自定义训练代码的自定义容器镜像。

在本教程中，我们将使用自定义容器来训练一个自定义模型。

创建一个私有的Docker仓库

您的第一步是在Google Artifact Registry中创建自己的Docker仓库。

In [ ]:
import os

DOCKER_REPOSITORY = f"{PROJECT_ID}-repo-unique"

! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

! gcloud artifacts repositories create {DOCKER_REPOSITORY} \
    --repository-format=docker \
    --location={LOCATION} \
    --description="Repository for TensorBoard Custom Training Job" \
    --quiet

! gcloud artifacts repositories list

### 配置身份验证到您的私有Docker仓库

在推送或拉取容器镜像之前，配置Docker使用`gcloud`命令行工具来对您的位置向Artifact Registry发起身份验证请求。

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if not IS_COLAB:
    ! gcloud auth configure-docker {LOCATION}-docker.pkg.dev --quiet

### 创建一个自定义容器镜像，并推送到您的私有Docker仓库

首先，您需要创建一个训练脚本文件和一个Docker文件。

创建一个存放所有训练代码的目录。

In [ ]:
PYTHON_PACKAGE_APPLICATION_DIR = "trainer"

!mkdir -p $PYTHON_PACKAGE_APPLICATION_DIR

#### 准备训练脚本

您的训练代码必须配置为将TensorBoard日志写入Cloud Storage存储桶，该存储桶的位置由Vertex AI Training自动通过预定义的环境变量`AIP_TENSORBOARD_LOG_DIR`提供。

通常可以通过将`os.environ['AIP_TENSORBOARD_LOG_DIR']`作为日志目录提供给开源TensorBoard日志写入API来实现。

例如，在TensorFlow 2.x中，您可以使用以下代码创建一个tensorboard_callback：

```
tensorboard_callback = tf.keras.callbacks.TensorBoard( 
  log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'], 
  histogram_freq=1) 
```

`AIP_TENSORBOARD_LOG_DIR`位于创建自定义训练作业时提供的`BASE_OUTPUT_DIR`中。

要为您的训练作业启用Cloud Profiler，请向训练脚本添加以下内容：

在顶层导入中添加cloud_profiler导入：

```
from google.cloud.aiplatform.training_utils import cloud_profiler
```

通过添加以下内容初始化cloud_profiler插件：

```
cloud_profiler.init()
```

In [ ]:
%%writefile trainer/task.py

import tensorflow as tf
import argparse
import os
import sys, traceback
from google.cloud.aiplatform.training_utils import cloud_profiler

"""Train an mnist model and use cloud_profiler for profiling."""

def _create_model():
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10),
        ]
    )
    return model


def main(args):
    print('Loading and preprocessing data ...')
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    print('Creating and training model ...')

    model = _create_model()
    model.compile(
      optimizer="adam",
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      metrics=["accuracy"],
    )

    # Initialize the profiler.
    print('Initialize the profiler ...')
        
    try:
        cloud_profiler.init()
    except:
        ex_type, ex_value, ex_traceback = sys.exc_info()
        print("*** Unexpected:", ex_type.__name__, ex_value)
        traceback.print_tb(ex_traceback, limit=10, file=sys.stdout)
    
    print('The profiler initiated.')

    log_dir = "logs"
    if 'AIP_TENSORBOARD_LOG_DIR' in os.environ:
      log_dir = os.environ['AIP_TENSORBOARD_LOG_DIR']

    print('Setting up the TensorBoard callback ...')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1)

    print('Training model ...')
    model.fit(
        x_train,
        y_train,
        epochs=args.epochs,
        verbose=0,
        callbacks=[tensorboard_callback],
    )
    print('Training completed.')

    print('Saving model ...')

    model_dir = "model"
    if 'AIP_MODEL_DIR' in os.environ:
      model_dir = os.environ['AIP_MODEL_DIR']
    tf.saved_model.save(model, model_dir)

    print('Model saved at ' + model_dir)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--epochs", type=int, default=100, help="Number of epochs to run model."
    )
    
    args = parser.parse_args()
    main(args)

准备Dockerfile

In [ ]:
%%writefile Dockerfile
# Specifies base image and tag
FROM us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-9:latest
WORKDIR /root

# Installs additional packages as you need.
RUN pip3 install google-cloud-aiplatform[cloud_profiler]
RUN pip3 install protobuf==3.20.2

# Copies the trainer code to the docker image.
RUN mkdir /root/trainer
COPY trainer/task.py /root/trainer/task.py

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

构建一个定制的容器镜像，并推送到您的私有Docker仓库。

In [ ]:
IMAGE_NAME = "tensorboard-custom-container"
IMAGE_URI = f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{DOCKER_REPOSITORY}/{IMAGE_NAME}"

! gcloud builds submit --project {PROJECT_ID} --region={LOCATION} --tag {IMAGE_URI} --timeout=60m --quiet

创建和运行自定义训练作业

使用自定义容器图像配置一个[自定义作业](https://cloud.google.com/vertex-ai/docs/training/create-custom-job)。

In [ ]:
JOB_NAME = "tensorboard-job-unique"

job = aiplatform.CustomContainerTrainingJob(
    display_name=JOB_NAME, container_uri=IMAGE_URI
)

### 运行自定义训练作业

接下来，您可以通过调用 `run` 方法来运行自定义作业以开始训练作业，参数如下：

- `args`：要传递给训练脚本的命令行参数。
   - `--epochs`：训练的时代数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 代表单节点训练）。
- `machine_type`：计算实例的机器类型。
- `tensorboard`：TensorBoard 实例。
- `service_account`：服务账户。
- `sync`：是否阻塞直到作业完成。

In [ ]:
base_output_dir = "{}/{}".format(BUCKET_URI, JOB_NAME)
MACHINE_TYPE = "n1-standard-4"
EPOCHS = 2
training_args = [
    "--epochs=" + str(EPOCHS),
]

job.run(
    args=training_args,
    replica_count=1,
    machine_type=MACHINE_TYPE,
    base_output_dir=base_output_dir,
    tensorboard=TENSORBOARD_INSTANCE_NAME,
    service_account=SERVICE_ACCOUNT,
)

查看 Cloud Profiler 仪表盘

当自定义作业状态切换为“运行”时，您可以通过 Google Cloud 控制台上的自定义作业页面或实验页面访问 Cloud Profiler 仪表盘。

谷歌云指南提供了[使用 Cloud Profiler 对模型训练性能进行分析](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-profiler)的详细说明，包括如何访问 Cloud Profiler 仪表盘和捕获一个性能分析会话。

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- Docker存储库
- 训练任务
- TensorBoard实例
- Cloud Storage存储桶

In [ ]:
delete_tensorboard = True
delete_bucket = False
delete_generated_files_after_execution = False

# Delete docker repository.
! gcloud artifacts repositories delete $DOCKER_REPOSITORY --project {PROJECT_ID} --location {LOCATION} --quiet

job.delete()

if delete_tensorboard:
    tensorboard.delete()

if delete_bucket and "BUCKET_URI" in globals():
    ! gsutil -m rm -r $BUCKET_URI

if delete_generated_files_after_execution:
    ! rm -rf $PYTHON_PACKAGE_APPLICATION_DIR Dockerfile Dockerfile